In [1]:
%%init_spark
launcher.conf.spark.app.name = "avinash"
launcher.jars = ["/app/setup/commons-1.0.1.jar"]
launcher.conf.spark.local.dir = "/dev/__delete__"
launcher.master = "local[20]"
launcher.conf.spark.driver.memory = "40g"
launcher.conf.spark.yarn.keytab = "/home/jioapp/aditya/couture.keytab"
launcher.conf.spark.yarn.principal = "couture@COUTURE.AI"
launcher.conf.spark.driver.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/krb5_new.conf"
launcher.conf.spark.executor.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/krb5_new.conf"
launcher.master = "local[20]"

In [2]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

Intitializing Scala interpreter ...

Spark Web UI available at http://10.166.181.219:4041
SparkContext available as 'sc' (version = 3.3.0, master = local[20], app id = local-1750312857358)
SparkSession available as 'spark'


import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._


In [3]:
def readP(path:String)={
    var df = spark.read.parquet(raw"hdfs://10.144.115.112:8020$path")
    df
}

readP: (path: String)org.apache.spark.sql.DataFrame


# INPUT DATA

In [4]:
var userData = readP("/data/ecomm/ajio/processed/prospectsDB")

userData: org.apache.spark.sql.DataFrame = [customerid: string, pincode: string ... 29 more fields]


In [5]:
userData.printSchema

root
 |-- customerid: string (nullable = true)
 |-- pincode: string (nullable = true)
 |-- userid: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- clienttype: string (nullable = true)
 |-- current_ts: string (nullable = true)
 |-- dateofbirth: string (nullable = true)
 |-- deviceid: string (nullable = true)
 |-- deviceregistered: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- jioprime: string (nullable = true)
 |-- kioskid: string (nullable = true)
 |-- lastlogin: string (nullable = true)
 |-- newlyregistered: string (nullable = true)
 |-- op_ts: string (nullable = true)
 |-- op_type: string (nullable = true)
 |-- ownerpkstring: string (nullable = true)
 |-- pos: string (nullable = true)
 |-- typePKString: string (nullable = true)
 |-- userprofile: string (nullable = true)
 |-- usercohortstringvalue: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- originaluid: string (nullable = true)
 |-- encryptedid: string (nullable = true)


In [4]:
var interactions = readP("/data/recommendations/ajio/sample/interactions")

25/06/19 11:31:19 WARN DataSource: Found duplicate column(s) in the data schema and the partition schema: `date`


interactions: org.apache.spark.sql.DataFrame = [user_id: string, product_id: string ... 12 more fields]


In [5]:
interactions.printSchema

root
 |-- user_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_sku_id: string (nullable = true)
 |-- interaction_type: string (nullable = true)
 |-- interaction_count: long (nullable = true)
 |-- interaction_context: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- date: date (nullable = true)
 |-- store: string (nullable = true)
 |-- pincode: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- booked_rev: double (nullable = true)



In [6]:
interactions
.select("interaction_type")
.distinct()
.show(100)

+----------------+
|interaction_type|
+----------------+
|   product_views|
|  product_clicks|
|     add_to_cart|
|    new_purchase|
|        wishlist|
|          orders|
|          return|
|        exchange|
+----------------+



# old input data

In [8]:
// var gaData = readP("/data/ecomm/ajio/processed/gaEventProspectCohort")

In [9]:
// gaData.printSchema

In [10]:
// gaData.select("event").distinct.show()

In [11]:
// // gaData = gaData
// .filter(col("event")==="TotalAddToCartPerDay")
// .withColumnRenamed("date","ga_date")

In [12]:
// gaData.printSchema

In [13]:
// gaData
// .select("ga_date")
// .distinct
// .show()

In [9]:
var productAttributes = readP("/data/ecomm/ajio/processed/productAttributesLegosFNL")

productAttributes: org.apache.spark.sql.DataFrame = [itemid: string, similargrouplevel: string ... 134 more fields]


In [10]:
productAttributes.printSchema

root
 |-- itemid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- additionalinformation1: string (nullable = true)
 |-- additionalinformation2: string (nullable = true)
 |-- additionalinformation3: string (nullable = true)
 |-- anklestyle: string (nullable = true)
 |-- audiotechnology: string (nullable = true)
 |-- basecolor: string (nullable = true)
 |-- basecurve: string (nullable = true)
 |-- belt: string (nullable = true)
 |-- bladematerial: string (nullable = true)
 |-- bladetype: string (nullable = true)
 |-- blousefabric: string (nullable = true)
 |-- bodytype: string (nullable = true)
 |-- bottomdescription: string (nullable = true)
 |-- bottomdetail: string (nullable = true)
 |-- bottomtype: string (nullable = true)
 |-- bottomwearfabric: string (nullable = true)
 |-- bottomweartype: string (nullable = true)
 |-- brushtype: string (nullable = true)
 |-- buttonclosure: string (nullable = true)
 |-- clasptype: string (nullable = true)
 |-- closure:

In [11]:
var interactionsProductInfoJoinedData = interactions.join(productAttributes,interactions("product_id")===productAttributes("itemid"),"inner")

interactionsProductInfoJoinedData: org.apache.spark.sql.DataFrame = [user_id: string, product_id: string ... 148 more fields]


In [17]:
// var userInteractionsJoinedData = userData.join(interactions,Seq("userid"),"inner")

In [18]:
// var interactionsCartInfroJoinedData = interactions.join(gaData,Seq("userid","productid"),"inner")

In [19]:
// interactionsCartInfroJoinedData.printSchema

# Lifetime order value

In [52]:
var LTOV = interactions
.filter(col("user_id").isNotNull && col("product_id").isNotNull && col("product_sku_id").isNotNull)
.filter(col("interaction_type")==="new_purchase" || col("interaction_type")==="return")
.withColumn("actual_price",when(col("interaction_type") === "return", -col("booked_rev")).otherwise(col("booked_rev")))
.groupBy("user_id")
.agg(
    sum("actual_price").as("LTOV")
)

LTOV
.show()

+--------------+------------------+
|       user_id|              LTOV|
+--------------+------------------+
| 8859543044100|            1287.0|
|10884536434692|            3572.0|
|11351330619396|             653.0|
|12152022958084|            9955.0|
| 9626263060484|            4309.0|
| 9920961118212|           2353.99|
|11608663949316|            2254.0|
|12046880899076|            1414.0|
| 9608230076420| 6277.599999999999|
| 9832901083140|             875.0|
| 8922543161348|             930.0|
|12156769370116|            1001.0|
|11282341363716|            1257.0|
|11851562319876|            750.75|
|11464364556292|            2397.0|
|11955915194372|3452.9999999999995|
|10059066998788|             780.0|
|12157564944388|           10219.0|
| 9107433783300|            3498.0|
|11192452710404|            3038.0|
+--------------+------------------+
only showing top 20 rows



LTOV: org.apache.spark.sql.DataFrame = [user_id: string, LTOV: double]


# Total value of return 

In [73]:
var TVR = interactions
.filter(col("user_id").isNotNull && col("product_id").isNotNull && col("product_sku_id").isNotNull)
.filter(col("interaction_type")==="return")
.select("user_id","product_id","product_sku_id")
.join(
    interactions
    .filter(col("user_id").isNotNull && col("product_id").isNotNull && col("product_sku_id").isNotNull)
    .filter(col("interaction_type")==="new_purchase"),
    Seq("user_id","product_id","product_sku_id"),
    "inner"
)
.select("user_id","product_id","product_sku_id","booked_rev")
.groupBy("user_id")
.agg(
    sum("booked_rev").as("TVR")
)

TVR.show()

+--------------+------------------+
|       user_id|               TVR|
+--------------+------------------+
|10033892196356|            1200.0|
|10580482195460|            688.29|
|10596110860292|            1119.0|
|10884536434692|             798.0|
|10951027687428|             399.0|
|11261266984964|            639.48|
|11608663949316|            1355.0|
| 8873360785412|13329.999999999998|
| 9122245804036|            1874.0|
| 9352047689732|           2466.36|
| 9454821605380|            3499.0|
| 9635426992132|            1396.0|
| 9816204247044|             639.0|
| 9903657779204|             709.0|
|10366932680708|1239.9899999999998|
|10381241057284|             939.3|
|10482543132676|             790.0|
|10523901558788|             847.0|
|10825243688964|             245.0|
|10866518327300|             944.0|
+--------------+------------------+
only showing top 20 rows



TVR: org.apache.spark.sql.DataFrame = [user_id: string, TVR: double]


# Total No of returns

In [55]:
var TNR = interactions
.filter(col("user_id").isNotNull && col("product_id").isNotNull && col("product_sku_id").isNotNull)
.filter(col("interaction_type")==="return")
.groupBy("user_id")
.count()
.withColumnRenamed("count","TNR")

TNR.show()

+--------------+---+
|       user_id|TNR|
+--------------+---+
| 9513080487940|  1|
| 9064323055620|  6|
|11478885400580|  1|
|12141426704388|  2|
| 9352264024068|  3|
|12107970084868|  1|
|12091246280708| 23|
|12126856445956|  1|
| 9876976369668|  1|
|11405960904708|  1|
| 9253427150852|  1|
| 9506571943940|  3|
| 8845305970692|  3|
|10042966736900|  1|
|10882744123396|  5|
| 8867037249540|  1|
| 9232119169028|  3|
| 9048606998532|  2|
|12139512823812|  5|
| 9840505061380|  4|
+--------------+---+
only showing top 20 rows



TNR: org.apache.spark.sql.DataFrame = [user_id: string, TNR: bigint]


# order frequency per week

In [65]:
val base = interactions
  .filter(col("user_id").isNotNull && col("product_id").isNotNull && col("product_sku_id").isNotNull)
  .filter(col("interaction_type") === "orders")
  .withColumn("ts", col("timestamp"))
  .withColumn("year", year(col("ts")))
  .withColumn("week", weekofyear(col("ts")))

val users = base.select("user_id").distinct()
val weeks = base.select("year", "week").distinct()

val userWeeks = users.crossJoin(weeks)

val ordersPerWeek = base
  .groupBy("user_id", "year", "week")
  .agg(size(collect_set("timestamp")).as("orders_per_week"))

val avgOrdersPerWeek = userWeeks
  .join(ordersPerWeek, Seq("user_id", "year", "week"), "left_outer")
  .na.fill(0, Seq("orders_per_week"))
  .groupBy("user_id")
  .agg(avg("orders_per_week").as("avg_orders_per_week"))

avgOrdersPerWeek.show(false)

+--------------+-------------------+
|user_id       |avg_orders_per_week|
+--------------+-------------------+
|10011090059268|1.5                |
|10014507630596|0.5                |
|10022775652356|0.5                |
|10053288624132|1.0                |
|10064022634500|0.5                |
|10071730716676|0.5                |
|10107498594308|1.5                |
|10157099843588|0.5                |
|10188190351364|0.5                |
|10188229836804|0.5                |
|10213787697156|0.5                |
|10215212089348|0.5                |
|10230620454916|0.5                |
|10241329528836|0.5                |
|10272744833028|0.5                |
|10318760148996|0.5                |
|10353391927300|0.5                |
|10356026343428|0.5                |
|10407012007940|0.5                |
|10467931324420|0.5                |
+--------------+-------------------+
only showing top 20 rows



base: org.apache.spark.sql.DataFrame = [user_id: string, product_id: string ... 15 more fields]
users: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: string]
weeks: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [year: int, week: int]
userWeeks: org.apache.spark.sql.DataFrame = [user_id: string, year: int ... 1 more field]
ordersPerWeek: org.apache.spark.sql.DataFrame = [user_id: string, year: int ... 2 more fields]
avgOrdersPerWeek: org.apache.spark.sql.DataFrame = [user_id: string, avg_orders_per_week: double]


# Average Interval Between consecutive orders

In [ ]:
interactions
.filter(col("interaction_type") === "orders" && col("user_id"))

In [72]:
val avgIntervalPerUser = interactions
.filter(col("user_id").isNotNull && col("product_id").isNotNull && col("product_sku_id").isNotNull)
.filter(col("interaction_type") === "orders")
.withColumn("ts", col("timestamp"))
.select("user_id", "ts")
.distinct()
.withColumn("prev_ts", lag("ts", 1).over(Window.partitionBy("user_id").orderBy("ts")))
.withColumn("interval_secs", unix_timestamp(col("ts")) - unix_timestamp(col("prev_ts")))
.where(col("interval_secs").isNotNull)
.groupBy("user_id")
.agg(avg("interval_secs").as("avg_interval_secs"))
.withColumn("avg_interval_days", round(col("avg_interval_secs") / 86400, 2))
.select("user_id", "avg_interval_secs", "avg_interval_days")

avgIntervalPerUser.show()

+--------------+-----------------+-----------------+
|       user_id|avg_interval_secs|avg_interval_days|
+--------------+-----------------+-----------------+
|10001856364548|         244355.0|             2.83|
|10002860539908|           1406.0|             0.02|
|10004011057156|          45042.0|             0.52|
|10004933443588|           5730.0|             0.07|
|10005146501124|         339162.0|             3.93|
|10006531375108|           5372.0|             0.06|
|10006621814788|         401653.0|             4.65|
|10007096655876|         179728.0|             2.08|
|10007181426692|         135420.0|             1.57|
|10007523131396|         306471.0|             3.55|
|10008816451588|          40001.4|             0.46|
|10008877989892|          88603.0|             1.03|
|10009154060292|         177413.0|             2.05|
|10010587856900|         134658.0|             1.56|
|10011090059268|          16524.5|             0.19|
|10011856535556|         434193.0|            

avgIntervalPerUser: org.apache.spark.sql.DataFrame = [user_id: string, avg_interval_secs: double ... 1 more field]


# Avg percentage discount

In [80]:
val avgDiscountPerUser = interactionsProductInfoJoinedData
.filter(col("user_id").isNotNull && col("product_id").isNotNull && col("product_sku_id").isNotNull)
.select("user_id","product_id","product_sku_id","booked_rev","mrp")
.withColumn(
  "discount",
  when(col("mrp").isNotNull && col("booked_rev").isNotNull && col("mrp") =!= 0,
    (lit(1.0) - (col("booked_rev").cast("double") / col("mrp").cast("double"))) * 100
  ).otherwise(null)
)
.groupBy("user_id")
.agg{
    avg("discount").as("avg_discount")
}
.select("user_id","avg_discount")

avgDiscountPerUser.show()

+--------------+------------------+
|       user_id|      avg_discount|
+--------------+------------------+
|10609029185540| 64.98939072446196|
| 9471382028292|              null|
| 8870738558980| 51.27540311999089|
| 9626709000196|   46.003600360036|
| 9114422476804|27.004611864518026|
| 9889221672964|49.004857836833835|
| 9496105517060| 63.24540567570946|
|10976041828356|             100.0|
|12005684871172| 59.97682755834796|
|11256766562308| 73.12276839543956|
| 9714236850180|-5.006130710999372|
|11296324386820|  74.9791492910759|
|11378487558148| 77.48203692967132|
| 9451729223684| 69.96413049122069|
| 9483958321156|61.565480365060594|
| 9245550215172| 60.52194506233311|
|10595497443332| 69.96996996996997|
| 9454821605380|47.984986353525976|
| 8872359067652| 35.50097333436544|
| 9558869508100|49.979629761956524|
+--------------+------------------+
only showing top 20 rows



avgDiscountPerUser: org.apache.spark.sql.DataFrame = [user_id: string, avg_discount: double]


# Avg bag value

In [18]:
interactionsProductInfoJoinedData
.filter(col("interaction_type")==="new_purchase")
.groupBy("user_id","timestamp")
.agg(
    collect_set("product_id").as("products_in_bag"),
    sum("mrp").as("bag_value")
)
.groupBy("user_id")
.agg(
    avg("bag_value").as("avg_bag_value")
)
.show(100)

+--------------+------------------+
|       user_id|     avg_bag_value|
+--------------+------------------+
| 9483958321156|            8131.0|
| 9960909897732|            5726.0|
| 9963159289860|            7594.0|
|11774836703236|            5267.6|
|11135405850628|            3615.0|
|11533232242692|            9294.0|
| 8978367741956|            2795.0|
|12124009168900|            3498.0|
| 9471107268612|           5450.83|
| 9606445400068|1031.7766666666666|
|10244843536388|            5298.0|
|12157546790916|           18795.0|
| 9992075935748|            3895.0|
| 8840453914628|            6344.0|
|11013851873284|            1544.0|
|10814141399044|            7007.5|
|12154976272388|            1049.0|
| 9626263060484|3891.3333333333335|
|11958677536772|            4648.5|
|11015252377604|            4646.0|
| 9000523595780|          8680.215|
|11263177588740|            1299.0|
|10931166969860|            1498.0|
|11673868173316|            6216.0|
| 9866799218692|            

# Avg cart to checkout time

In [88]:
import org.apache.spark.sql.functions._

val AvgCartToCheckoutTime = interactions
  .filter(col("interaction_type") === "add_to_cart")
  .select("user_id", "product_id", "timestamp")
  .withColumnRenamed("timestamp", "cart_time")
  .join(
    interactions
      .filter(col("interaction_type") === "new_purchase")
      .select("user_id", "product_id", "timestamp")
      .withColumnRenamed("timestamp", "purchase_time"),
    Seq("user_id", "product_id"),
    "inner"
  )
  .withColumn("cart_to_checkout_secs", unix_timestamp(col("purchase_time")) - unix_timestamp(col("cart_time")))
  .withColumn("cart_to_checkout_hours", round(col("cart_to_checkout_secs") / 3600.0, 2))
  .withColumn("cart_to_checkout_days", round(col("cart_to_checkout_secs") / 86400.0, 2))
  .select("user_id", "product_id", "cart_time", "purchase_time", "cart_to_checkout_hours", "cart_to_checkout_days")
  .groupBy("user_id")
  .agg(
      avg("cart_to_checkout_hours").as("avg_cart_to_checkout_hours"),
      avg("cart_to_checkout_days").as("avg_cart_to_checkout_days")
  )

AvgCartToCheckoutTime.show(false)

+--------------+--------------------------+-------------------------+
|user_id       |avg_cart_to_checkout_hours|avg_cart_to_checkout_days|
+--------------+--------------------------+-------------------------+
|10001885036548|20.99                     |0.87                     |
|10010282491908|13.67                     |0.57                     |
|10016305348612|23.23                     |0.9700000000000001       |
|10027175084036|32.18636363636363         |1.3436363636363635       |
|10033892196356|23.13                     |0.96                     |
|10034499190788|7.8                       |0.33                     |
|10157930479620|16.4                      |0.68                     |
|10192824664068|10.08                     |0.42                     |
|10204382101508|12.85                     |0.54                     |
|10215212089348|22.209999999999997        |0.93                     |
|10229182955524|32.714444444444446        |1.3588888888888888       |
|10274929868804|11.8

import org.apache.spark.sql.functions._
AvgCartToCheckoutTime: org.apache.spark.sql.DataFrame = [user_id: string, avg_cart_to_checkout_hours: double ... 1 more field]


# total no of categories purchased

In [ ]:
val totalNoCategories = interactionsProductInfoJoinedData
.select("user_id", "l1code", "l2code", "brickname", "interaction_count")
.filter(col("interaction_count") =!= 0)
.groupBy("user_id")
.agg(
  collect_set("l1code").as("l1codes"),
  collect_set("l2code").as("l2codes"),
  collect_set("brickname").as("bricks")
)
.withColumn("num_l1codes", size(col("l1codes")))
.withColumn("num_l2codes", size(col("l2codes")))
.withColumn("num_bricks", size(col("bricks")))
.select("user_id","l1codes","l2codes","bricks", "num_l1codes", "num_l2codes", "num_bricks")

totalNoCategories.show()

Intitializing Scala interpreter ...

# attribute affinity

In [22]:
val totalNoCategories = interactionsProductInfoJoinedData
.select("userid", "l1code", "l2code", "brickname", "quantity")
.filter(col("quantity") =!= 0)
.groupBy("userid")
.agg(
  collect_set("l1code").as("l1codes"),
)
.withColumn("num_l1codes", size(col("l1codes")))
.select("userid","l1codes", "num_l1codes")

totalNoCategories.show()

25/06/17 13:50:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


org.apache.spark.sql.AnalysisException:  Column 'l2codes' does not exist. Did you mean one of the following? [l1codes, num_l1codes, userid];

# brand affinity

In [16]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import scala.math._

val weightsMap = Map(
  "product_views" -> 1,
  "product_clicks" -> 5,
  "add_to_cart" -> 8,
  "wishlist" -> 7,
  "orders" -> 10,
  "new_purchase" -> 10,
  "return" -> -5,
  "exchange" -> -3
)

val broadcastWeightsMap = spark.sparkContext.broadcast(weightsMap)

val getWeight = udf((interactionType: String) => broadcastWeightsMap.value.getOrElse(interactionType, 0))

val lambda = 0.05
val expDecay = udf((daysAgo: Long) => java.lang.Math.exp(-lambda * daysAgo))

val topBrandAffinity = interactionsProductInfoJoinedData
  .filter(col("user_id").isNotNull && col("brandname").isNotNull && col("interaction_count") =!= 0)
  .withColumn("weight", getWeight(col("interaction_type")))
  .withColumn("days_ago", datediff(current_date(), col("date")))
  .withColumn("decay", expDecay(col("days_ago")))
  .withColumn("raw_score", col("interaction_count") * col("weight"))
  .withColumn("decayed_score", col("raw_score") * col("decay"))
  .groupBy("user_id", "brandname")
  .agg(sum("decayed_score").as("total_weighted_score"))
  .withColumn("brand_affinity", struct(
    col("brandname").as("brand"),
    org.apache.spark.sql.functions.round(col("total_weighted_score"), 2).as("affinity_score")
  ))
  .select("user_id", "brand_affinity")

topBrandAffinity.show(false)

25/06/19 11:56:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/06/19 11:56:35 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/06/19 11:56:35 WARN SparkConf: The configuration key 'spark.yarn.principal' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.principal' instead.
25/06/19 11:56:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/06/19 11:56:35 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/06/19 11:56:35 WARN SparkConf: The configuration key 'spark.yarn.principal' has been deprecated as of Spark 3.0 and may be removed in the future. Pleas

import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import scala.math._
weightsMap: scala.collection.immutable.Map[String,Int] = Map(return -> -5, product_clicks -> 5, new_purchase -> 10, exchange -> -3, product_views -> 1, wishlist -> 7, add_to_cart -> 8, orders -> 10)
broadcastWeightsMap: org.apache.spark.broadcast.Broadcast[scala.collection.immutable.Map[String,Int]] = Broadcast(19)
getWeight: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5837/181309467@2bdfbb77,IntegerType,List(Some(class[value[0]: string])),Some(class[value[0]: int]),None,false,true)
lambda: Double = 0.05
expDecay: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$5838/83791645@7269cd1f,DoubleType,List...
